In [1]:
import tensorflow as tf
import numpy
import matplotlib.pyplot as plt
print(tf.__version__)

2.4.0


In [2]:
dataset = tf.data.Dataset.range(10)
for val in dataset:
    print(val.numpy())

0
1
2
3
4
5
6
7
8
9


## tf.data.dataset.window
- 시계열 데이터를 다룰 때 사용하면 매우 유용
- 시계열 데이터를 다룰 때 다음 함수와 비슷한 것들을 직접 정의하여 sequence를 만들어주어야 하는 번거로움이 있음

```
def make_sequence(data, n):
    X, y = list(), list()
    
    for i in range(len(data)):
        _X = data.iloc[i:(i + n), :-1]
        if(i + n) < len(data):
            X.append(np.array(_X))
            y.append(data.iloc[i + n, -1])
        else:
            break
            
    return np.array(X), np.array(y)
```
tf.data를 사용하면 여러 줄로 구성되어 있는 위의 코드가 단 하나의 함수로 해결됨.

```
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1)
for window_dataset in dataset:
  for val in window_dataset:
    print(val.numpy(), end=" ")
  print()
```



In [3]:
# dataset.window의 첫 번째 인자는 window size이고, 두 번째는 shift 크기를 전달
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift = 1)
for window_dataset in dataset:
    for val in window_dataset:
        print(val.numpy(), end = " ")
    print()

0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 
6 7 8 9 
7 8 9 
8 9 
9 


결과에서 window_size = 5만큼의 데이터를 얻다가, 끝 부분에서 [6, 7, 8, 9], [7, 8, 9], ... 의 원치않는 결과를 얻고 있습니다.
이는 가져오려는 window_size가 데이터셋의 크기를 초과했기 때문에 그렇습니다.

이를 방지하기 위해 drop_remainder = True 인자를 사용합니다.

In [4]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
for window_dataset in dataset:
  for val in window_dataset:
    print(val.numpy(), end=" ")
  print()

0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 


* for-loop를 2중으로 사용하는 이유는 dataset.window가 Tensor가 아닌 Dataset을 반환하기 때문
* 이는 flat_map 함수를 사용해서 window_dataset을 flat해주어 바로 사용할 수 있다
* 즉 원래 같은 경우 5 -> 4 -> 3 -> 처럼 iter 형식으로 받을 수 있었는데, flat_map을 사용하면 [5, 4, 3, 2, 1]로 바로 받을 수 있다.

In [5]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift = 1, drop_remainder = True)
dataset = dataset.flat_map(lambda window: window.batch(5))
for window in dataset:
    print(window.numpy())

[0 1 2 3 4]
[1 2 3 4 5]
[2 3 4 5 6]
[3 4 5 6 7]
[4 5 6 7 8]
[5 6 7 8 9]


In [6]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift = 1, drop_remainder = True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:])) # 마지막 전까지, 마지막꺼만으로 나눔
for x, y in dataset:
    print(x.numpy(), y.numpy())

[0 1 2 3] [4]
[1 2 3 4] [5]
[2 3 4 5] [6]
[3 4 5 6] [7]
[4 5 6 7] [8]
[5 6 7 8] [9]


In [7]:
# shuffle도 가능
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift = 1, drop_remainder = True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
dataset = dataset.shuffle(buffer_size=10)
for x, y in dataset:
    print(x.numpy(), y.numpy())

[1 2 3 4] [5]
[5 6 7 8] [9]
[3 4 5 6] [7]
[4 5 6 7] [8]
[2 3 4 5] [6]
[0 1 2 3] [4]


In [10]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift = 1, drop_remainder = True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
dataset = dataset.shuffle(buffer_size=10)
dataset = dataset.batch(2).prefetch(1) # prefetch는 cpu놀리는걸 방지
for x, y in dataset:
    print("x = ", x.numpy())
    print("y = ", y.numpy())

x =  [[1 2 3 4]
 [3 4 5 6]]
y =  [[5]
 [7]]
x =  [[2 3 4 5]
 [0 1 2 3]]
y =  [[6]
 [4]]
x =  [[5 6 7 8]
 [4 5 6 7]]
y =  [[9]
 [8]]
